## Naive Bayes Document Classification

## Teacher Led Introduction

In this lesson, we'll take a look at another machine learning classification algorithm: Naive Bayes. As you may have guessed, this algorithm is based on Bayes Theorem for conditional probability. The intuition is that we can split our [training] data along various features from the dataset and observe the conditional probability of the various categories we are looking to predict. From this, we can then calculate the probability of a test case being from each of the various classes, with the highest probability being the prediction that we'll make.


For example, suppose we are using an API to gather articles from a news website and grabbing phrases from two different types of articles:  music and politics. Imagine that only some of our articles have their category (music or politics). The naive bayes classifier could be very appropriate for generating a prediction in this scenario.

## Classifying Articles 

-------------------------------
### Here are our articles
#### Music Articles:

* 'the song was popular'
* 'band leaders disagreed on sound'
* 'played for a sold out arena stadium'

#### Politics Articles

* 'world leaders met lask week'
* 'the election was close'
* 'the officials agreed on a compromise'
--------------------------------------------------------
Let's try and predict one example phrase:


* "world leaders agreed to fund the stadium"

To do this, we'll use a bag of words representation and assume that words are independent of one another.

In [1]:
from collections import defaultdict
import numpy as np
music = ['the song was popular',
         'band leaders disagreed on sound',
         'played for a sold out arena stadium']

politics = ['world leaders met lask week',
            'the election was close',
            'the officials agreed on a compromise']

test_statement = 'world leaders agreed to fund the stadium'

In [2]:
#labels : 'music' 'politics'
#features: words
test_statement_2 = 'officials met at the arena'

<img src ="./resources/naive_bayes_icon.png">

### Another way of looking at it
<img src = "./resources/another_one.png">

## So, in the context of our problem......



## $ P(politics | phrase) = \frac{P(phrase|politics)P(politics)}{P(phrase)}$

## $ P(politics) = \frac{ \# politics}{\# all\ articles} $

*where phrase is our test statement*

Estimating the parameters of our model (using Maximum a Posteriori):

<img src = "./resources/solving_theta.png" width="400">

### How should we calculate P(politics)?

This is essentially the distribution of the probability of either type of article. We have three of each type of article, therefore, we assume that there is an equal probability of either article

In [3]:
p_politics = len(politics)/(len(politics) + len(music))

In [4]:
p_politics

0.5

In [5]:
p_music = len(music)/(len(politics) + len(music))

In [6]:
p_music

0.5

### How do you think we should calculate: $ P(phrase | politics) $ ?

> We need to break the phrases down into individual words.






## $ P(phrase | politics) = \prod_{i=1}^{d} P(word_{i} | politics) $

### We need to make a *Naive* assumption.

> The Naive assumption is assuming independence for each word. This is inherently false; English has many linguistic rules and the probability of a given word is certainly not invariant based on the words surrounding it.



### $ P(word_{i} | politics) = \frac{\#\ of\ word_{i}\ in\ politics\ art.} {\#\ of\ total\ words\ in\ politics\ art.} $

### Can you foresee any issues with this?

> A: We can't have a probability of 0. If we see a new word that was not in the training set, a probability of 0 will negate all other information. This behavior is problematic. How should we account for unseen cases?




## Laplace Smoothing
## $ P(word_{i} | politics) = \frac{\#\ of\ word_{i}\ in\ politics\ art. + \alpha} {\#\ of\ total\ words\ in\ politics\ art. + \alpha d} $

## $ P(word_{i} | music) = \frac{\#\ of\ word_{i}\ in\ music\ art. + \alpha} {\#\ of\ total\ words\ in\ music\ art. + \alpha d} $

This correction process is called Laplace smoothing:
* d : number of features (in this instance total number of vocabulary words)
* $\alpha$ can be any number greater than 0 (it is usually 1)


#### Now let's find this calculation

<img src="./resources/IMG_0041.jpg">

In [ ]:
# p(phrase|politics)

In [11]:
def vocab_maker(category):
    """returns the vocabulary for a given type of article"""
    vocab_category = set()
    for art in category:
        words = art.split()
        for word in words:
            vocab_category.add(word)
    return vocab_category
        
voc_music = vocab_maker(music)
voc_pol = vocab_maker(politics)
total_vocabulary = voc_music.union(voc_pol)


In [12]:
voc_music

{'a',
 'arena',
 'band',
 'disagreed',
 'for',
 'leaders',
 'on',
 'out',
 'played',
 'popular',
 'sold',
 'song',
 'sound',
 'stadium',
 'the',
 'was'}

In [13]:
voc_pol

{'a',
 'agreed',
 'close',
 'compromise',
 'election',
 'lask',
 'leaders',
 'met',
 'officials',
 'on',
 'the',
 'was',
 'week',
 'world'}

In [14]:
voc_all = voc_music.union(voc_pol)

In [15]:
voc_all

{'a',
 'agreed',
 'arena',
 'band',
 'close',
 'compromise',
 'disagreed',
 'election',
 'for',
 'lask',
 'leaders',
 'met',
 'officials',
 'on',
 'out',
 'played',
 'popular',
 'sold',
 'song',
 'sound',
 'stadium',
 'the',
 'was',
 'week',
 'world'}

In [16]:
total_vocab_count = len(voc_all)
total_music_count = len(voc_music)
total_politics_count = len(voc_pol)

In [ ]:
#P(politics | leaders agreed to fund the stadium)

In [17]:
def find_number_words_in_category(phrase,category):
    statement = phrase.split()
    str_category=' '.join(category)
    cat_word_list = str_category.split()
    word_count = defaultdict(int)
    for word in statement:
        for art_word in cat_word_list:
            if word == art_word:
                word_count[word] +=1
            else:
                word_count[word]
    return word_count

In [18]:
test_music_word_count = find_number_words_in_category(test_statement,music)


In [19]:
test_music_word_count

defaultdict(int,
            {'world': 0,
             'leaders': 1,
             'agreed': 0,
             'to': 0,
             'fund': 0,
             'the': 1,
             'stadium': 1})

In [20]:
test_politic_word_count = find_number_words_in_category(test_statement,politics)

In [21]:
test_politic_word_count

defaultdict(int,
            {'world': 1,
             'leaders': 1,
             'agreed': 1,
             'to': 0,
             'fund': 0,
             'the': 2,
             'stadium': 0})

In [22]:
def find_likelihood(category_count,test_category_count,alpha):
    num = np.product(np.array(list(test_category_count.values())) + alpha)
    denom = (category_count + total_vocab_count*alpha)**(len(test_category_count))
    
    return num/denom

In [23]:
likelihood_m = find_likelihood(total_music_count,test_music_word_count,1)

In [24]:
likelihood_p = find_likelihood(total_politics_count,test_politic_word_count,1)

In [25]:
print(likelihood_m)
print(likelihood_p)

4.107740405680756e-11
1.748875897714495e-10


### $ P(politics | article) = P(politics) x \prod_{i=1}^{d} P(word_{i} | politics) $

#### Deteriming the winner of our model:

<img src = "./resources/solvingforyhat.png" width= "400">

In [29]:
classes = ['Music', 'Politics']
probabilities = [likelihood_m, likelihood_p]
prediction_class_idx = np.argmax([likelihood_m, likelihood_p]) #Choose the higher index of the probabilities
prediction = classes[prediction_class_idx]
prediction

'Politics'

Many times, the probabilities we end up are exceedingly small, so we can transform them using logs to save on computation speed

### $ log(P(politics | article)) = log(P(politics)) + \sum_{i=1}^{d}log( P(word_{i} | politics)) $







### Different Types of Naive Bayes Classifiers

Multinomial Naive Bayes Classifier: this is the example we just did! It is essentially a collection of a Bernoulli Naive Bayes Classifier. This classifer cannot handle negative values!



The Bernoulli Naive Bayes Classifier: used when your features are binary (0 or 1). In the context of a text based classification task, this would be whether or not a word appears in a document at all and calculating the probability of it occuring.

<img src = "./resources/bernoulli_nb_formula.svg">


There is also the Gaussian Naive Bayes Classifier, which assumes that the features that you are predicting based off of are normally distributed.

<img src = "./resources/normal_posterior.png">